## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

- датасет состоит из `21525` строк и `12` столбцов;

- столбцы `days_employed` и `total_income` содержат пропущенные значения; 

- в ячейках датасета встречаются целые числа, вещественные числа и строки.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
data.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
print('Пропуски в столбце days_employed:', data['days_employed'].isna().sum())
print('Пропуски в столбце total_income:', data['total_income'].isna().sum())

Пропуски в столбце days_employed: 2174
Пропуски в столбце total_income: 2174


В столбцах `days_employed` и `total_income` имеется **2174** пропусков. Посмотрев на первые 50 строк датасета, заметно, что в столбце `days_employed` значения записаны со знаком "минус". Встречаются и положительные значения, но они по какой-то причине аномально большие (и встречаются только в строках со значением "пенсионер" столбца `income_type`). Названные столбцы имеют пропущенные значения (**NaN**), необходимо их заполнить для дальнейшей работы с данными. Можно заменить пропуски дохода медианой, но взятой по 2-3 характеристикам клиента.

In [12]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()

data_copy = data.copy()
# Соберу сводную таблицу с группами по типу занятости, образованию и семейному статусу
table = data_copy.pivot_table(index=['income_type', 'family_status'], columns='education', values='total_income', aggfunc='median')
table

education                                     высшее      начальное  \
income_type     family_status                                         
безработный     гражданский брак       202722.511368            NaN   
                женат / замужем                  NaN            NaN   
в декрете       женат / замужем                  NaN            NaN   
госслужащий     в разводе              168966.283322            NaN   
                вдовец / вдова         171832.375773            NaN   
                гражданский брак       174889.695518   73471.521358   
                женат / замужем        167170.112319  148393.772010   
                не женат / не замужем  189052.122478  190912.178349   
компаньон       в разводе              199740.203074  173719.870634   
                вдовец / вдова         188552.695079            NaN   
                гражданский брак       194902.662418  170077.891115   
                женат / замужем        208874.400817  133572.776907   
                не женат / не замужем  196715.578710  136798.905143   
пенсионер       в разводе              147991.177090  106412.862533   
                вдовец / вдова         121593.923004  103954.539610   
                гражданский брак       161906.119642   72062.859516   
                женат / замужем        149986.600709  114348.307573   
                не женат / не замужем  136040.181456   83957.535989   
предприниматель гражданский брак       499163.144947            NaN   
сотрудник       в разводе              169553.747930  122164.630609   
                вдовец / вдова         146829.125455  149500.865270   
                гражданский брак       164328.469420  122114.452405   
                женат / замужем        168096.668601  131201.719463   
                не женат / не замужем  157648.978017  124303.014695   
студент         не женат / не замужем   98201.625314            NaN   

education                              неоконченное высшее        среднее  \
income_type     family_status                                               
безработный     гражданский брак                       NaN            NaN   
                женат / замужем                        NaN   59956.991984   
в декрете       женат / замужем                        NaN   53829.130729   
госслужащий     в разводе                              NaN  145833.125801   
                вдовец / вдова                         NaN  116228.591335   
                гражданский брак             159647.976722  134409.451334   
                женат / замужем              165325.246377  137010.946239   
                не женат / не замужем        155079.532422  144240.653068   
компаньон       в разводе                    193535.278277  163649.774285   
                вдовец / вдова               249638.675937  142098.238590   
                гражданский брак             167211.393638  158556.924920   
                женат / замужем              179964.390359  160915.448553   
                не женат / не замужем        176604.151178  154147.380971   
пенсионер       в разводе                              NaN  113978.152522   
                вдовец / вдова               108896.366143  118697.279928   
                гражданский брак              93232.027392  115290.905764   
                женат / замужем              144462.810869  114158.577028   
                не женат / не замужем        131932.062171  115080.782380   
предприниматель гражданский брак                       NaN            NaN   
сотрудник       в разводе                    198330.710953  136095.570156   
                вдовец / вдова               162906.336478  138617.149666   
                гражданский брак             164890.370611  137289.176687   
                женат / замужем              144398.769467  137131.915406   
                не женат / не замужем        145919.113211  132878.497248   
студент         не женат / не замужем                  NaN            NaN   

ed

In [13]:
def my_fillna_func (income_type, family_status, education):
    try:
        return table.loc[(income_type, family_status)][education]
    except:
        return 0
    
print(my_fillna_func('сотрудник', 'гражданский брак', 'высшее')) # Проверка работы функции

164328.46942027909


In [14]:
data.head(50) # Таблица до применения функции (содержит пропуски)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [15]:
data.apply(lambda row: my_fillna_func(row['income_type'], row['family_status'], row['education']), axis=1)

0        168096.668601
1        137131.915406
2        137131.915406
3        137131.915406
4        115290.905764
             ...      
21520    158556.924920
21521    114158.577028
21522    137289.176687
21523    137131.915406
21524    137131.915406
Length: 21525, dtype: float64

In [32]:
data['total_income'] = data['total_income'].fillna(data.groupby(['income_type','gender', 'education'])['total_income'].transform('median'))

data['days_employed'] = data['days_employed'].fillna(data.groupby(['income_type','gender', 'education'])['total_income'].transform('median'))

Проверим, что пропуски убраны.

In [33]:
print('Пропуски в столбце days_employed:', data['days_employed'].isna().sum())
print('Пропуски в столбце total_income:', data['total_income'].isna().sum())

Пропуски в столбце days_employed: 1
Пропуски в столбце total_income: 1


**Вывод**

Обнаруженные пропуски в данных могли появиться из-за ошибок при записи или транспортировании файла, либо данных о рабочем стаже и, соответственно, доходах людей с пропусками просто не имелось у банка. Для заполнения пропусков использовался метод **.median()**, т.к. среднее значение могло получиться слишком большим из-за аномально высоких значений в данных. 

### Замена типа данных

In [34]:
data = data.fillna(0)
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Данные в столбцах с типом данных **float64** переведены в тип **int64** методом **.astype()**.

### Обработка дубликатов

In [35]:
data['education'].value_counts() # В столбце 'education' замечены дубликаты

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [36]:
data['education'] = data['education'].str.lower() # Убираем дубликаты с помощью приведения всех символов столбца в нижний регистр
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

**Вывод**

Столбец с данными об образовании имел множество дубликатов, т.к. одни и те же значения были записаны по-разному, с разной высотой символов. Причиной этому может быть человеческий фактор, отсутствие инструкции о единообразном заполнении данного столбца. Дубликаты убраны вручную при помощи метода **.str.lower()** библиотеки Pandas. Данный метод выбран в соответствии с пройденным уроком о нахождении дубликатов.

### Лемматизация

In [37]:
# Импорт библиотеки pymystem3
from pymystem3 import Mystem
m = Mystem()

purpose_unique = data['purpose'].unique() # Получение списка с уникальными значениями столбца 'purpose' 
purpose_unique_str = ','.join(purpose_unique) # Объединение списка в строку 

lemm = m.lemmatize(purpose_unique_str) # Лемматизируем слова из строки и считаем частоту появления слов
from collections import Counter
Counter(lemm)

Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         ',': 37,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

Чаще всего встречаются слова **автомобиль**, **образование**, **недвижимость**, **жилье** и **покупка**. Последние три можно объединить в одну категорию "недвижимость". Также можно выделить слово **свадьба**. Это основные категории причин взятия кредита в банке.

In [38]:
# Определение функции, которая лемматизирует каждую ячейку столбца 'purpose'
def purp_lemm(purpose):
    if 'жилье' in ' '.join(m.lemmatize(purpose)):
        return 'недвижимость'
    elif 'недвижимость' in ' '.join(m.lemmatize(purpose)):
        return 'недвижимость'
    elif 'образование' in ' '.join(m.lemmatize(purpose)):
        return 'образование'
    elif 'автомобиль' in ' '.join(m.lemmatize(purpose)):
        return 'автомобиль'
    elif 'свадьба' in ' '.join(m.lemmatize(purpose)):
        return 'свадьба'
    return purpose

data['lemm_purpose'] = data['purpose'].apply(purp_lemm) # Добавление столбца с лемматизированными вариантами категорий
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemm_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


**Вывод**

Получены уникальные значения столбца `purpose`, которые затем были лемматизированы. Частота появления лемматизированных значений посчитана с помощью **Counter**. Самые частые существительные из получившегося словаря определены вручную как основные категории. 

### Категоризация данных

In [39]:
data['dob_years'].value_counts() # Уникальные значения в столбце возраста

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [40]:
data['children'].value_counts() # Уникальные значения в столбце количества детей

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [41]:
data['total_income'].min() # Минимальный ежемесячный доход

0

In [42]:
data['total_income'].max() # Максимальный ежемесячный доход

2265604

In [43]:
def age_group(age): # Функция для категоризации по возрасту
    if age <= 35:
        return 'молодые'
    if age <= 55:
        return 'средних лет'
    return 'возрастные'

def children(kids): # Функция для категоризации по количеству детей в семье
    if kids == 0:
        return 'детей нет'
    if kids <= 3:
        return 'несколько детей'
    return 'много детей'

def money(income): # Функция для категоризации по уровню дохода
    if income <= 95000:
        return 'низкий доход'
    if income <= 135000:
        return 'доход ниже среднего'
    if income <= 155000:
        return 'средний доход'
    if income <= 215000:
        return 'доход выше среднего'
    return 'высокий доход'
    
data['age_group'] = data['dob_years'].apply(age_group) # Добавление столбца с категорией клиента по возрасту
data['family_size'] = data['children'].apply(children) # Добавление столбца с категорией клиента по колиеству детей
data['income_group'] = data['total_income'].apply(money) # Добавление столбца с категорией клиента по уровню дохода

In [44]:
data['age_group'].value_counts()

средних лет    10461
молодые         6695
возрастные      4369
Name: age_group, dtype: int64

In [45]:
data['family_size'].value_counts()

детей нет          14149
несколько детей     7250
много детей          126
Name: family_size, dtype: int64

In [46]:
data['income_group'].value_counts()

доход ниже среднего    5602
доход выше среднего    5203
высокий доход          4331
низкий доход           3894
средний доход          2495
Name: income_group, dtype: int64

In [47]:
# КОД РЕВЬЮЕРА

pd.qcut(data['total_income'], q=5)

0        (215396.2, 2265604.0]
1          (98616.0, 127716.8]
2         (127716.8, 160654.8]
3        (215396.2, 2265604.0]
4         (127716.8, 160654.8]
                 ...          
21520    (215396.2, 2265604.0]
21521     (127716.8, 160654.8]
21522        (-0.001, 98616.0]
21523    (215396.2, 2265604.0]
21524        (-0.001, 98616.0]
Name: total_income, Length: 21525, dtype: category
Categories (5, interval[float64]): [(-0.001, 98616.0] < (98616.0, 127716.8] < (127716.8, 160654.8] < (160654.8, 215396.2] < (215396.2, 2265604.0]]

In [48]:
# КОД РЕВЬЮЕРА

pd.qcut(data['total_income'], 5, ['низкий','ниже среднего','средний','выше среднего', 'высокий'])

0              высокий
1        ниже среднего
2              средний
3              высокий
4              средний
             ...      
21520          высокий
21521          средний
21522           низкий
21523          высокий
21524           низкий
Name: total_income, Length: 21525, dtype: category
Categories (5, object): ['низкий' < 'ниже среднего' < 'средний' < 'выше среднего' < 'высокий']

**Вывод**

Категоризация данных может быть произведена разными способами: по возрасту клиента, по количеству его детей, а также по уровню дохода. Были категоризированы столбцы именно с этими данными, потому что остальные столбцы уже имеют в себе определенный набор значений (по ним удобно сортировать). А столбцы с численными начениями, как, например, уровень дохода, требовалось привести в вид, удобный для дальнейшей работы. Теперь имеется общее представление о том, сколько клиентов имеют большие семьи, много или мало зарабатывают и т.д.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [49]:
 # Здесь и далее процент должников считается с нахождением среднего по бинарному столбцу 'debt'
    
data.groupby('family_size').agg({'debt': 'mean'}).sort_values(by='debt')

,debt
family_size,
детей нет,0.075129
несколько детей,0.091862
много детей,0.095238


In [50]:
# КОД РЕВЬЮЕРА

data.groupby('family_size')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
family_size,,,
детей нет,14149,1063,7.51%
много детей,126,12,9.52%
несколько детей,7250,666,9.19%


**Вывод**

Между наличием у клиента детей и возвратом кредита в срок прослеживается зависимость: чем меньше детей, тем меньше процент должников.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [51]:
data.groupby('family_status').agg({'debt': 'mean'}).sort_values(by='debt')

,debt
family_status,
вдовец / вдова,0.065625
в разводе,0.071130
женат / замужем,0.075202
гражданский брак,0.092890
не женат / не замужем,0.097405


In [52]:
data.groupby('family_status')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
family_status,,,
в разводе,1195,85,7.11%
вдовец / вдова,960,63,6.56%
гражданский брак,4177,388,9.29%
женат / замужем,12380,931,7.52%
не женат / не замужем,2813,274,9.74%


**Вывод**

Зависимость странная: вдовцы и раведенные люди возвращают кредит в срок чаще, чем люди в гражданском браке и женатые. Но по какой-то причине среди неженатых людей самый большой процент должников.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [55]:
data.groupby('income_group').agg({'debt': 'mean'}).sort_values(by='debt')

,debt
income_group,
высокий доход,0.069730
низкий доход,0.080123
доход ниже среднего,0.082114
средний доход,0.086573
доход выше среднего,0.086681


In [56]:
data.groupby('income_group')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
income_group,,,
высокий доход,4331,302,6.97%
доход выше среднего,5203,451,8.67%
доход ниже среднего,5602,460,8.21%
низкий доход,3894,312,8.01%
средний доход,2495,216,8.66%


**Вывод**

Четкой зависимости нет: клиенты с высоким ежемесячным доходом и клиенты низким доходом возвращают долг чаще, чем остальные клиенты.

- Как разные цели кредита влияют на его возврат в срок?

In [57]:
data.groupby('lemm_purpose').agg({'debt': 'mean'}).sort_values(by='debt')

,debt
lemm_purpose,
недвижимость,0.072140
свадьба,0.079216
образование,0.091994
автомобиль,0.093395


In [58]:
data.groupby('lemm_purpose')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
lemm_purpose,,,
автомобиль,4315,403,9.34%
недвижимость,10840,782,7.21%
образование,4022,370,9.20%
свадьба,2348,186,7.92%


**Вывод**

Долги в срок чаще возвращают клиенты, которые взяли кредит на покупку недвижимости или на проведение свадьбы, чем те клиенты, которые брали кредит на образование или покупку автомобиля.

## Шаг 4. Общий вывод

Была поставлена следующая задача: выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Чтобы иметь возможность анализировать данные, проведена предобработка полученных от банка данных: убраны пропуски в данных и дубликаты, данные были категоризированы с добавлением новых столбцов с категориями клиентов по количеству детей в семье, по уровню их ежемесячных доходов, а также по возрасту. Между количеством детей и возвратом кредита в срок найдета четкая зависимость: **чем больше детей в семье, тем больше процент невозврата кредита в срок**. `9,52% невозврата у клиентов с большим количеством детей, в то время как среди клиентов без детей процент невозврата составил 7,51%`. А вот между семейным положением клиентов и процентом воврата кредита в срок чёткой зависимости **не найдено**. С другой стороны, если не брать в расчет неженатых людей, то чаще возвращают кредит те, кто находится в разводе или является вдовцом (вдовой) - `6,56%`. Неженатые люди же вовращают долги в срок **чаще всех** (`9,74%`). По уровню дохода самыми ответственными клиентами оказались те, кто имеют высокий уровень дохода: `процент невозврата 7%`, а самыми безответственными - клиенты с с доходами выше среднего (`8,68% невозвратов`). Самые надежные клиенты - это те люди, которые берут кредит на операции с недвижимостью (`процент невозврата 7,21%`), а самые безответственные - те, кто обращается в банк с целью получить деньги на покупку автомобиля (`9,34% невозврата`).